# Estimation of piping failure probability (Bligh's model)

In this example, we will demonstrate the application of various reliability methods to estimate the probability of failure of a levee due to piping mechanism. The failure mechanism is characterized here using the simple model of Bligh.

### Define model

First, we import the necessary package:

In [1]:
from streams import *

The limit state function according to the piping model of Bligh is defined as follows:

$Z = \frac{m \cdot L}{c_{creep}} -\Delta H$

where: <br>

$\Delta H$ is the difference between the water level on the levee and the water level in the polder [m] <br>
$L$ is the seepage path length (m) <br>
$m$ represents model uncertainty (-) <br>
$c_{creep}$ is the creep factor (-)

The parameter $\Delta H$ represents the imposed load, while $L$ stands for the strength of the levee. The creep factor value for very fine-grained sand is $18$.

In [2]:
def bligh(m, L, c_creep, delta_H):
    return m * L / c_creep - delta_H

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [3]:
project = ReliabilityProject()
project.model = bligh

We assume the following distributions for the parameters $m$, $L$, $c_{creep}$ and $\Delta H$:

In [4]:
project.variables["m"].distribution = DistributionType.log_normal
project.variables["m"].mean = 1.76
project.variables['m'].deviation = 1.69

project.variables["L"].distribution = DistributionType.normal
project.variables["L"].mean = 50
project.variables["L"].deviation = 2.5

project.variables["c_creep"].distribution = DistributionType.deterministic
project.variables["c_creep"].mean = 18

project.variables["delta_H"].distribution = DistributionType.gumbel
project.variables["delta_H"].shift = 0.53
project.variables["delta_H"].scale = 0.406

### Perform reliability calculations with FORM

We start with the reliability method `form`. We choose the calculation settings: `relaxation_factor`, `maximum_iterations` and `variation_coefficient`. The reliability analysis is executed using `project.run()`, and the results are accessed from `project.design_point`.

In [5]:
project.settings.reliability_method = ReliabilityMethod.form

project.settings.relaxation_factor = 0.75
project.settings.maximum_iterations = 50
project.settings.variation_coefficient = 0.02

project.run()

def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")
    
    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")

    print(f"Model runs = {dp.total_model_runs}")
    
read_results(project.design_point)

Beta = 1.627239311154229
Probability of failure = 0.051843140823976314
m: alpha = 0.833063979847028, x = 0.4243756225382517
L: alpha = 0.04866412691132183, x = 49.80202954911724
c_creep: alpha = 0.0, x = 18.0
delta_H: alpha = -0.5510319484688609, x = 1.174386594416648
Converged (convergence = 0.009028332172632802 < 0.02)
Model runs = 24


### Perform reliability calculations with Crude Monte Carlo

We now conduct the reliability analysis using the `crude_monte_carlo` method.

In [6]:
project.settings.reliability_method = ReliabilityMethod.crude_monte_carlo

project.settings.minimum_samples = 1000
project.settings.maximum_samples = 50000
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.644230562447786
Probability of failure = 0.05006429309218156
m: alpha = 0.8108514521738569, x = 0.4321182572644983
L: alpha = 0.047182916234664, x = 49.80605101775388
c_creep: alpha = 0.0, x = 18.0
delta_H: alpha = -0.583346976441243, x = 1.215495930680213
Converged (convergence = 0.01999932321915781 < 0.02)
Model runs = 47440


### Perform reliability calculations with Numerical Integration
We now conduct the reliability analysis using the `numerical_integration` method.

In [7]:
project.settings.reliability_method = ReliabilityMethod.numerical_integration

project.settings.stochast_settings["m"].intervals = 50
project.settings.stochast_settings["L"].intervals = 20
project.settings.stochast_settings["delta_H"].intervals = 50

project.run()

read_results(project.design_point)

Beta = 1.6605659897086176
Probability of failure = 0.04840032215651717
m: alpha = 0.8140532694038095, x = 0.42568296760382024
L: alpha = 0.049152780499551324, x = 49.79594641100708
c_creep: alpha = 0.0, x = 18.0
delta_H: alpha = -0.578706556678021, x = 1.2167100214923094
Not converged (convergence = nan > 0.02)
Model runs = 50001
